In [378]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

Hacer lista con los equipos por pais

In [379]:
LISTA_RONDAS = ["Final", "Final", "Semifinal", "Semifinal", "Cuartos", "Cuartos", "Octavos", "Octavos"]
LISTA_NUM_LINK = ["torneo=1048", "torneo=1047", "torneo=1046", "torneo=1045", 
                "torneo=1044", "torneo=1043", "torneo=1074", "torneo=1089", "libertadores"]
                

In [380]:
def revisar_equipo(l, equipo):
    for i in l:
        eq = i.replace(",", "")
        eq = eq.replace(".", "")
        eq = eq.replace(" ", "")
        if eq[:3].upper() == equipo.upper():
            return i
    return "Dep. Cali"

In [381]:
locales = []
visitas = []
gol_locales = []
gol_visitas = []
resultados = []
goles = []
dif_goles = []
dias = []
anos = []
fases = []
for num in LISTA_NUM_LINK:
    cont = 0
    url = f"https://www.promiedos.com.ar/{num}"
    r = rq.get(url)
    sopa = bs(r.content, "html.parser")
    lista_fases = sopa.find_all('div', id='partoff')
    final_unica = sopa.find_all('div', id='partoff1')

    if num == "libertadores":
        url = "https://www.promiedos.com.ar/libertadores=faseprevia"
        r = rq.get(url)
        sopa = bs(r.content, "html.parser")

    lista_grupos = sopa.find_all('div', class_ = 'grupo')
    if final_unica != []:
        equ = final_unica[0].find_all('td', class_='game-t1')
        equ1 = equ[1].text ; equ2 = equ[2].text
        locales.append(equ1)
        visitas.append(equ2)

        res1 = final_unica[0].find_all('td', class_='game-r1')
        res1 = (res1[1].text)[0]
        gol_locales.append(int(res1))
        res2 = final_unica[0].find_all('td', class_='game-r2')
        res2 = (res2[1].text)[0]
        gol_visitas.append(int(res2))

        if int(res1) == int(res2):
            resultados.append("Empate")
            dif_goles.append(0)

        elif int(res1) > int(res2):
            resultados.append("Local")
            dif_goles.append(int(res1) - int(res2))

        elif int(res1) < int(res2):
            resultados.append("Visita")
            dif_goles.append(int(res2) - int(res1))

        goles.append(int(res1) + int(res2))


        dia = final_unica[0].find_all('tr', class_='diapart')
        dia = dia[0].text
        dia = dia.split(" ")
        if len(dia) <= 4:
            dia.append("2022")
        anos.append(dia[4])
        ano = dia[4]
        dia = dia[1] + " " + dia[2] + " " + dia[3] + " " + dia[4]
        dias.append(dia)
        fases.append("Final")
        cont = 2

    for data in lista_fases:
        lista_partidos = data.find_all('div', id='fixturein')
        fase = LISTA_RONDAS[cont]
        for datos in lista_partidos:
            equ = datos.find_all('td', class_='game-t1')
            equ1 = equ[1].text ; equ2 = equ[2].text
            locales.append(equ1)
            visitas.append(equ2)

            res1 = datos.find_all('td', class_='game-r1')
            res1 = (res1[1].text)[0]
            gol_locales.append(int(res1))
            res2 = datos.find_all('td', class_='game-r2')
            res2 = (res2[1].text)[0]
            gol_visitas.append(int(res2))

            if int(res1) == int(res2):
                resultados.append("Empate")
                dif_goles.append(0)

            elif int(res1) > int(res2):
                resultados.append("Local")
                dif_goles.append(int(res1) - int(res2))

            elif int(res1) < int(res2):
                resultados.append("Visita")
                dif_goles.append(int(res2) - int(res1))

            goles.append(int(res1) + int(res2))


            dia = datos.find_all('tr', class_='diapart')
            dia = dia[0].text
            dia = dia.split(" ")
            if len(dia) <= 4:
                dia.append("2022")
            anos.append(dia[4])
            ano = dia[4]
            dia = dia[1] + " " + dia[2] + " " + dia[3] + " " + dia[4]
            dias.append(dia)

            fases.append(fase)

        cont += 1
        if cont == 8:
            break

    for data in lista_grupos:
        equipos = data.find('table', id='posiciones')
        equipos = equipos.find_all('td', align = 'left')
        equipos = [i.text for i in equipos]
        partidos = data.find_all('div', id = 'fixgrupo')
        for datos in partidos:
            horario = datos.find_all('div', class_ = 'diahoragr')
            res = datos.find_all('div', class_ = 'grtr')

            for j in range(2):
                eq1 = res[j].find('div', class_ = 'greq1')
                eq1 = revisar_equipo(equipos, eq1.text)
                locales.append(eq1)
                eq2 = res[j].find('div', class_ = 'greq2')
                eq2 = revisar_equipo(equipos, eq2.text)
                visitas.append(eq2)

                resultado = res[j].find('div', class_ = 'grres4')
                resultado = resultado.text
                resultado = resultado.split('-')
                res1 = int(resultado[0])
                res2 = int(resultado[1])
                gol_locales.append(res1)
                gol_visitas.append(res2)

                if res1 == res2:
                    resultados.append("Empate")
                    dif_goles.append(0)

                elif int(res1) > int(res2):
                    resultados.append("Local")
                    dif_goles.append(int(res1) - int(res2))

                elif int(res1) < int(res2):
                    resultados.append("Visita")
                    dif_goles.append(int(res2) - int(res1))

                goles.append(int(res1) + int(res2))
                anos.append(ano)
                dia = horario[j].text
                dia = dia.replace(",", "") ; dia = dia.split(" ")
                dia = dia[3]
                dias.append(dia)
                fases.append("Grupos")
        

In [382]:
df_libertadores = pd.DataFrame()
df_libertadores["Local"] = locales
df_libertadores["Visita"] = visitas
df_libertadores["Gol_local"] = gol_locales
df_libertadores["Gol_visita"] = gol_visitas
df_libertadores["Resultado"] = resultados
df_libertadores["Año"] = anos
df_libertadores["Fase"] = fases
df_libertadores["Goles"] = goles
df_libertadores["Diferencia_goles"] = dif_goles
df_libertadores["Dia"] = dias

In [383]:
df_libertadores

,Local,Visita,Gol_local,Gol_visita,Resultado,Año,Fase,Goles,Diferencia_goles,Dia
0,Nacional (P),San Lorenzo,1,1,Empate,2014,Final,2,0,06 de Agosto 2014
1,San Lorenzo,Nacional (P),1,0,Local,2014,Final,1,1,13 de Agosto 2014
2,Nacional (P),Def. Sporting,2,0,Local,2014,Semifinal,2,2,22 de Julio 2014
3,Def. Sporting,Nacional (P),1,0,Local,2014,Semifinal,1,1,29 de Julio 2014
4,San Lorenzo,Bolivar,5,0,Local,2014,Semifinal,5,5,23 de Julio 2014
...,...,...,...,...,...,...,...,...,...,...
1111,Sp. Cristal,U. Catolica,1,1,Empate,2022,Grupos,2,0,4/5
1112,Flamengo,U. Catolica,3,0,Local,2022,Grupos,3,3,17/5
1113,Sp. Cristal,Talleres (C),0,0,Empate,2022,Grupos,0,0,17/5
1114,Flamengo,Sp. Cristal,2,1,Local,2022,Grupos,3,1,24/5
